# CNN hyperparameter tuning 
- stride 1 filter 3x3
- stride 2 filter 3x3
- stride 3 filter 3x3
- stride 1 filter 2x2
- stride 1 filter 4x4

In [2]:
#get data
#show random picture
#split the data test and train
#take the image data and normalize the data
#label encode
#define CNN
#prediction and actual accuracy
#Confusion matrix visulization

import warnings
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from PIL import Image
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras.layers
from keras.utils.np_utils import to_categorical
from keras import Sequential
from keras.regularizers import L2
#get data
label_data=pd.read_csv('E:\python\python_projects\deepfaceTensorflow\DeepLearning\CNN\datasets\cifar10Labels.csv',index_col=0)
import keras.callbacks
# from warnings
from mlxtend.evaluate import confusion_matrix,scoring
from mlxtend.plotting import plot_confusion_matrix
from skimage.transform import resize
from  vis.visualization import visualize_cam,visualize_saliency,overlay
from vis.utils import utils

In [3]:
print(label_data)

            label
id               
1            frog
2           truck
3           truck
4            deer
5      automobile
...           ...
49996        bird
49997        frog
49998       truck
49999  automobile
50000  automobile

[50000 rows x 1 columns]


In [34]:

ytrain,ytest=train_test_split(label_data['label'],test_size=0.2,random_state=42)
print(ytrain.shape,ytest.shape)

(40000,) (10000,)


In [20]:
temp=[]

for i in ytrain.index:
    img_path=os.path.join('E:\python\python_projects\deepfaceTensorflow\DeepLearning\CNN\datasets\cifar10',f'{i}.png')
    img=np.array(imageio.imread(img_path)).astype('float32')
    temp.append(img)

xtrain=np.stack(temp)
print(xtrain.shape)
temp=[]
for i in ytest.index:
    img_path=os.path.join('E:\python\python_projects\deepfaceTensorflow\DeepLearning\CNN\datasets\cifar10',f'{i}.png')
    img=np.array(imageio.imread(img_path)).astype('float32')
    temp.append(img)
xtest=np.stack(temp)
print(xtest.shape)


xtrain=xtrain/255.
xtest=xtest/255.

(40000, 32, 32, 3)
(10000, 32, 32, 3)


In [35]:
lb=LabelEncoder()

ytraint=lb.fit_transform(ytrain)
ytrain=to_categorical(ytraint)
ytestt=lb.fit_transform(ytest)
ytest=to_categorical(ytestt)

print(ytrain,ytest)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]] [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# stride 1 filter 3X3

In [49]:

num_classes=10
model=Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu',kernel_regularizer=L2(0.01),input_shape=(32,32,3),name='conv1'),
    keras.layers.Normalization(name='BN1'),
    keras.layers.MaxPool2D(pool_size=(2,2),name='maxpool1'),
    keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu',kernel_regularizer=L2(0.01),input_shape=(32,32,3),name='conv2'),
    keras.layers.Normalization(name='BN2'),
    keras.layers.MaxPool2D(pool_size=(2,2),name='maxpool2'),
    keras.layers.Flatten(name='Flat'),
    keras.layers.Dense(num_classes,activation='softmax',name='pred_layer'),
    ])
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(),metrics=['accuracy'])
cpfile=r'cifar10.hdf5'
cb_checkpoint=keras.callbacks.ModelCheckpoint(cpfile,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
model.fit(xtrain,ytrain,validation_split=0.2,epochs=5,callbacks=[cb_checkpoint])




Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 32, 32, 32)        896       
                                                                 
 BN1 (Normalization)         (None, 32, 32, 32)        65        
                                                                 
 maxpool1 (MaxPooling2D)     (None, 16, 16, 32)        0         
                                                                 
 conv2 (Conv2D)              (None, 16, 16, 64)        18496     
                                                                 
 BN2 (Normalization)         (None, 16, 16, 64)        129       
                                                                 
 maxpool2 (MaxPooling2D)     (None, 8, 8, 64)          0         
                                                                 
 Flat (Flatten)              (None, 4096)             

In [50]:
#evaluate the model

pred_ytrain=model.predict(xtrain)
pred_ytrain=np.argmax(pred_ytrain,axis=1)
print(pred_ytrain)
print(f'train accuracy: {scoring(ytraint,pred_ytrain,metric="accuracy")*100}')
pred_ytest=model.predict(xtest)
pred_ytest=np.argmax(pred_ytest,axis=1)
print(pred_ytest)
print(f'test accuracy: {scoring(ytestt,pred_ytest,metric="accuracy")*100}')

1250/1250 [==============================] - 2s 1ms/step
[6 7 5 ... 8 8 8]
train accuracy: 60.1075
313/313 [==============================] - 1s 2ms/step
[5 8 0 ... 8 7 3]
test accuracy: 56.58


# stride 3 filter 3x3